email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# Setup local model
import os
from langchain_ollama import ChatOllama

model_name="granite4:3b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

# Setup cloud model
# model="gtp-5-nano"

In [3]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

In [4]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [5]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [6]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [7]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [8]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    model=model,
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Please check my inbox")]},
    context=EmailContext(),
    config=config
)

from pprint import pprint
pprint(response['messages'][-1].content)

("I'm sorry, but authentication failed. Please ensure you've entered the "
 'correct email address and password, or check with your account administrator '
 'for any issues that might be preventing access to your inbox. If you have '
 'further questions or need assistance with anything else, feel free to let me '
 'know!')


In [13]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="julie@example.com, password123")]},
    context=EmailContext(),
    config=config
)

from pprint import pprint

pprint(f'''Authenticated : {response['authenticated']}''')

'Authenticated : True'


In [14]:
print(response['messages'][-1].content)

An email from **Jane** has been found in your inbox. The subject is "Let's Grab Coffee?" and the message body reads:  
*"Hi Julie, I'm going to be in town next week and was wondering if we could grab a coffee? - best, Jane (jane@example.com)"*


In [15]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Draft an email to meet next week")]},
    context=EmailContext(),
    config=config
)

In [16]:
pprint(response['messages'][-1].tool_calls[0]['args']['body'])

('Hi Jane,\n'
 '\n'
 'It sounds like a great idea! Next Friday afternoon works for me. How about '
 'we meet at the usual spot, 3 PM? Please confirm if that suits you.\n'
 '\n'
 'Looking forward to catching up!\n'
 'Best,\n'
 'Julie')


In [17]:
from langgraph.types import Command

response = agent.invoke(
    Command (
        resume={"decisions": [{"type": "approve"}]} # or "reject"
    ),
    config=config
)

In [18]:
pprint(response['messages'][-1].content)

('An email has been sent to **Jane** with the subject "Re: Let\'s Grab '
 'Coffee?" and a message body confirming your plan to meet on Friday afternoon '
 'at the usual spot for 3 PM. The details are:\n'
 '\n'
 "**Subject:** Re: Let's Grab Coffee?  \n"
 '**Message Body:** Hi Jane,\n'
 '\n'
 'It sounds like a great idea! Next Friday afternoon works for me. How about '
 'we meet at the usual spot, 3 PM? Please confirm if that suits you.\n'
 '\n'
 'Looking forward to catching up!\n'
 '\n'
 'Best,\n'
 'Julie')
